# ECG Analysis Example

<div class="alert alert-block alert-info">
This example notebook illustrates how to further analyze and plot processed ECG data (i.e. outputs generated from a processing pipeline as depicted in <code>ECG_Processing_Example.ipynb</code>).
</div>

In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np

import biopsykit as bp
import biopsykit.signals.ecg as ecg
from biopsykit.signals.ecg import EcgProcessor

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close('all')
plt.rcParams['figure.figsize'] = (10,5)
sns.set(style='ticks')

### Load Data

**Subject Conditions**

In [ ]:
dict_condition = bp.io.load_subject_condition_list("../example_data/condition_list.csv")

**Heart Rate Phase Dict**

Load absolute and normalized heart rate exports. Which kind of data you need depends on the research question you are interested in and the analysis approach you choose

In [ ]:
dict_phase = pd.read_excel("../example_data/hr_phase_export_sample.xlsx", index_col="time", sheet_name=None)
dict_phase_norm = pd.read_excel("../example_data/hr_phase_export_sample_normalized.xlsx", index_col="time", sheet_name=None)

### Rearrange Data

**Split 'Phase dict' into 'Subphase dict'** (*optional*):

If your data/protocol/etc. is divided into multiple phases, each consisting of two or more subphases you might want to analyze separately or compare among each other, you can split the 'Phase dict' into a 'Subphase dict' using `bp.utils.data_processing.split_subphases()`

The output is `dict_subph`: a Nested dictionary with heart rate data per Phase and Subphase, respectively
* keys = Phase names
* values = dictionary with Heart Rate data per Subphase:
    * keys = Subphase names
    * values = Heart Rate data (pandas DataFrame)

If your subphases have fixed lengths and each subphase begins right after the previous subphase you can simply pass a list of subphase names and durations as parameter:

In [ ]:
dict_subph = bp.utils.data_processing.split_dict_into_subphases(dict_phase, subphases={'Start': 60, 'Middle': 180, 'End': 120})

If your subphases have fixed lengths, but each subphase does *not* begin right after the previous subphase you can pass the relative start and end times (in seconds) of each subphase as list of tuples:

In [ ]:
dict_subph = bp.utils.data_processing.split_dict_into_subphases(dict_phase, subphase_names=['Start', 'Middle', 'End'], subphase_times=[(0, 60), (80, 260), (280, 340)])

If the last subphase does not have fixed length (e.g. because the last subphase was a *Feedback* phase with variable length), but you want all the data, you first need to find the phase with the longest recording and pass this as the last subphase duration. The shorter intervals are automatically cut to their maximal length

In [ ]:
dict_subph = bp.utils.data_processing.split_dict_into_subphases(dict_phase, subphases={'Start': 60, 'Middle': 180, 'End': 0})

**Split HR Phase Dict into Conditions**

In [ ]:
dict_groups = bp.utils.data_processing.split_subject_conditions(dict_phase, dict_condition)

**Split 'Phase dict' of each condition into 'Subphase dict'**

Same as splitting a simple 'Phase dict' without different conditions, just set `is_group_dict=True`

In [ ]:
dict_subph_groups = bp.utils.data_processing.split_subject_conditions(dict_subph, dict_condition)

#### Compute Parameter

**Mean Heart Rate PER SUBJECT during each Phase/Subphase**

$\Rightarrow$ These parameters can then be exported and/or used to compute statistics and for plotting

Only phases - no group dict

In [ ]:
bp.utils.data_processing.mean_per_subject_dict(dict_phase, dict_levels=["phase", "subject"], param_name="HR")

Phases and subphases - no group dict

In [ ]:
bp.utils.data_processing.mean_per_subject_dict(dict_subph, dict_levels=["phase", "subphase", "subject"], param_name="HR")

Only phases - group dict

In [ ]:
bp.utils.data_processing.mean_per_subject_dict(dict_groups, dict_levels=["condition", "phase", "subject"], param_name="HR")

Phases and subphases - group dict

In [ ]:
bp.utils.data_processing.mean_per_subject_nested_dict(dict_subph_groups, param_name="HR", is_group_dict=True).head()

**Mean and Standard Error of Heart Rate during each Subphase** (for all data and split into conditions)

In [ ]:
# compute 'mse dataframe' for all data
mean_se = bp.utils.data_processing.mean_se_nested_dict(dict_subph)
mean_se

In [ ]:
# compute 'mse dataframe' for each condition individually
mean_se_groups = bp.utils.data_processing.mean_se_nested_dict(dict_subph_groups, is_group_dict=True)
mean_se_groups